In [1]:
import json
import numpy as np
import random

class CharacterSwap:
    def __init__(self):
        pass

    def insert_misspelling(self, word, idx):
        chars = [*word]
        n_chars = len(word)
        if idx < n_chars - 1:
            temp_char = chars[idx]
            chars[idx] = chars[idx + 1]
            chars[idx + 1] = temp_char
        else:
            temp_char = chars[idx]
            chars[idx] = chars[idx - 1]
            chars[idx - 1] = temp_char
        return "".join(chars)

class CharacterDuplicate:
    def __init__(self):
        pass

    def insert_misspelling(self, word, idx):
        chars = [*word]
        n_chars = len(word)
        chars.insert(idx, chars[idx])
        return "".join(chars)

class CharacterDelete:
    def __init__(self):
        pass
    def insert_misspelling(self, word, idx):
        chars = [*word]
        n_chars = len(word)
        del chars[idx]
        return "".join(chars)


class CharacterKeyboard:
    def __new__(cls):
        if not hasattr(cls, "instance"):
            cls.instance = super(CharacterKeyboard, cls).__new__(cls)
        return cls.instance

    def __init__(self):
        with open('./keybord_es.json', 'r') as f:
            self.keyboard_misspellings = json.load(f)

    def insert_misspelling(self, word, idx):
        chars = [*word]
        n_chars = len(word)
        misspellings = self.keyboard_misspellings[chars[idx]]
        chars[idx] = random.choice(misspellings)

        return "".join(chars)

class CharacterMS:
    def __init__(self, char_max = 2, char_probs = [0.89,0.11]):
        self.char_max = char_max,
        self.char_probs = char_probs

    def get_idxes(self, word):
        [num_idxes]  = random.choices([1,2], weights= self.char_probs)
        self.chars = [*word]
        self.n_chars = len(word)
        idxes = random.sample(list(range(self.n_chars)), k = num_idxes)
        return idxes

    def generate_misspellings(self, word):
        idxs = self.get_idxes(word)
        ms_types = random.choices([1,2,3,4], k= len(idxs))
        for idx, ms_type in zip(idxs, ms_types):
            word = self.insert_misspelling(word, idx, ms_type)
        
        return word

    def insert_misspelling(self, word, idx, ms_type):
        if ms_type == 1:
            return CharacterSwap().insert_misspelling(word, idx)
        elif ms_type == 2:
            return CharacterDuplicate().insert_misspelling(word, idx)
        elif ms_type == 3:
            return CharacterDelete().insert_misspelling(word, idx)
        elif ms_type == 4:
            return CharacterKeyboard().insert_misspelling(word, idx)     

In [79]:
import numpy as np
import random
import random
import spacy
from nltk.corpus import stopwords

nlp = spacy.load('es_core_news_md')
random.seed(42)

## tokenizar

def tokenize(text):
    tokens = [str(token) for token in nlp(text)]
    return tokens

class MisspellingsAug:
    def __init__(self, aug_pct = 0.5, tok_pct = 0.1):
        self.aug_pct = aug_pct
        self.tok_pct = tok_pct
        self.load_natural_misspellings()

    def load_natural_misspellings(self):
        with open('errores_ortograficos.json', "r", encoding= "utf-8") as f:
            errores_ortograficos = json.load(f)

        self.word2missp = {}

        for e in errores_ortograficos:
            word = e['palabra']
            misspellings = e['errores_ortograficos']
            self.word2missp[word] = misspellings
    
    def get_words_with_natural_misspellings(self):
        return [ k for k,v in self.word2missp.items() if len(v) > 0 ]

    def aug_texts(self, texts , stop_words):
        aug_size = np.ceil(self.aug_pct * len(texts))
        #print("aug_size:", aug_size)
        texts_aug = random.sample(texts, k = int(aug_size))
        all_texts = texts.copy()
        for text in texts_aug:
            tokens = tokenize(text)
            idx2token  = { str(idx):token for idx, token in enumerate(tokens) if not token in stop_words}
            num_tokens = len(idx2token)
            num_misspellings = int(np.ceil(tok_pct * num_tokens))

            words_wnm = self.get_words_with_natural_misspellings()
            idx_tok_wnm = [ int(idx) for idx, tok  in idx2token.items() if tok in words_wnm]
            
            idx_ton_not_wnm = [ int(idx) for idx, tok  in idx2token.items() if not tok in words_wnm]

            types_misspellings = random.choices([1,2], k = num_misspellings)
            
            for ms in types_misspellings:
                print("ms:", ms)
                
                if len(idx_tok_wnm) == 0:
                    ms = 2

                if ms == 1:
                    
                    idx_elem = random.choice(list(range(len( idx_tok_wnm ))))
                    print("idx_tok_wnm:", idx_tok_wnm)
                    print("idx_elem:", idx_elem)
                    idx = idx_tok_wnm[idx_elem]
                    word = tokens[idx]
                    tokens[idx] = self.insert_natural_misspelling(word)
                    del idx_tok_wnm[idx_elem]

                if ms == 2:
                    idx_tokens = idx_tok_wnm + idx_ton_not_wnm
                    idx_elem = random.choice(list(range(len( idx_tokens ))))
                    print("idx_tok_wnm:", idx_tok_wnm)
                    print("idx_elem:", idx_elem)
                    idx = idx_tokens[idx_elem]
                    word = tokens[idx]
                    tokens[idx] = self.insert_synthetic_misspelling(word)
                    if len(idx_tok_wnm) == 0 or len(idx_tok_wnm) <= idx_elem:
                        del idx_ton_not_wnm[idx_elem - len(idx_tok_wnm)]
                    else:
                        del idx_tok_wnm[idx_elem]      

            new_text = " ".join(tokens)
            all_texts.append(new_text)
        return all_texts

    def insert_misspelling(self, word, ms_type):
        idx = random.choice(list(range(len(word))))
        
        if ms_type == 1:
            return CharacterSwap().insert_misspelling(word, idx)
        elif ms_type == 2:
            return CharacterDuplicate().insert_misspelling(word, idx)
        elif ms_type == 3:
            return CharacterDelete().insert_misspelling(word, idx)
        elif ms_type == 4:
            return CharacterKeyboard().insert_misspelling(word, idx)
        
    def insert_synthetic_misspelling(self, word , char_max = 2, char_probs = [0.89,0.11]):
        [num_idxes]  = random.choices(list(range(1,char_max + 1)), weights= char_probs, k=1)
        chars = [*word]
        n_chars = len(word)
        #idxes = random.sample(list(range(n_chars)), k = num_idxes)
        ms_char_types = random.choices([1,2,3,4], k = num_idxes)
        
        for ms_type in ms_char_types:
            word = self.insert_misspelling(word, ms_type)
        return word

    def insert_natural_misspelling(self, word):
        misspellings = self.word2missp[word]
        
        if len(misspellings) > 0:
            misspelling = random.choice(misspellings)
            return misspelling
        return word

texts = ["por que motivos procede el retiro total", "cuando puedo pagar el autoseguro"]
aug_pct = 0.5
tok_pct  = 0.2

## Obtener stopwords
stop_words = stopwords.words('spanish')

marks = ["?",".", ",","¿"]
stop_words.extend(marks)

question_words = ['qué','que','quién','quien', 'cuál','cual', 'dónde','donde', 'cuándo','cuando', 'cómo','como', 'por qué','por que','porqué','porque', 
'cuánto','cuanto','cuantos']
stop_words = [ sw for sw in stop_words if not sw in question_words ]

misspellingsAug = MisspellingsAug(aug_pct, tok_pct )

misspellingsAug.aug_texts(texts , stop_words)

ms: 1
idx_tok_wnm: [1, 3]
idx_elem: 1


['por que motivos procede el retiro total',
 'cuando puedo pagar el autoseguro',
 'por que motivos prosede el retiro total']

In [98]:
import pandas as pd
df = pd.read_csv('test.csv')
intent_examples = {}
intent2label = {}
for row in df.iterrows():
    intent_name = row[1]['intent_name']
    example = row[1]['examples']
    label = row[1]['label']

    if not intent_name in intent_examples:
        intent_examples[intent_name] = [example]
    else:
        intent_examples[intent_name].append(example)

    if not intent_name in intent2label:
        intent2label[intent_name] = label

intent2label

{'retiro_total__requisitos_documentos_justificacion': 25,
 'procedimiento_tramites_fechas': 5,
 'confirmar_requerir_mas_ayuda': 3,
 'horario_atencion_aera': 43,
 'matricula__horarios': 13,
 'despedida': 0,
 'pago_autoseguro_procedimiento': 15,
 'reincorporacion__formato_solicitud': 35,
 'solicitud_correo_institucional_procedimiento': 44,
 'carnet_universitario__solicitud': 11,
 'pago_autoseguro__fechas': 14,
 'retiro_total__procedimiento': 28,
 'reincorporacion__procedimiento': 32,
 'matricula__cronograma': 12,
 'constancia_de_estudios__formato_solicitud': 41,
 'matricula__procedimiento_alumno_regular': 9,
 'retiro_total__formato_solicitud': 27,
 'reserva_matricula__procedimiento': 29,
 'matricula_procedimiento': 8,
 'cursos_horarios': 19,
 'constancia_de_notas__procedimiento': 39,
 'cambio_seccion': 18,
 'retiro_parcial__procedimiento': 21,
 'constancia_de_estudios__procedimiento': 42,
 'perdida_turno_matricula': 16,
 'matricula__procedimiento_alumno_ingresante': 10,
 'denegar_mas_ayu

In [99]:
aug_pct = 0.6
tok_pct  = 0.3

## Obtener stopwords
stop_words = stopwords.words('spanish')

stop_words.extend(["?",".", ",","¿"])

question_words = ['qué','que','quién','quien', 'cuál','cual', 'dónde','donde', 'cuándo','cuando', 'cómo','como', 'por qué','por que','porqué','porque', 
'cuánto','cuanto','cuantos']

stop_words = [ sw for sw in stop_words if not sw in question_words ]

## Generador de errores ortográficos

misspellingsAug = MisspellingsAug(aug_pct, tok_pct )

dict_data = {'intent_name': [], 'examples' : [], "label": []}

for intent_name, examples in intent_examples.items():
    normalized_examples = [ e.lower() for e in examples]
    print(normalized_examples)
    aug_examples = misspellingsAug.aug_texts(normalized_examples, stop_words)
    dict_data['examples'].extend(aug_examples)
    dict_data['intent_name'].extend( [intent_name] * len(aug_examples))
    intent_label = intent2label[intent_name]
    dict_data['label'].extend( [intent_label] * len(aug_examples))
    
dict_data

['debo justificar la razon del retiro toal?', 'debo presentar documentos para justificar los motivos de retiro total?', '¿es necesario justificar con documentos el motivo de retiro total?', 'es necesario presentar documentos para justificar el retiro total?', 'de que manera se tiene que justificar el retiro total?', 'he sufrido un accidente y quiero retirme del ciclo pero aun no tengo una constancia de salud podria solicitarlo sin la constancia', '¿se requiere documentos para justificar el retiro total?', 'sera necesario presentar documentos para justificar el retiro total?', 'debo justificar mis motivos para solicitar mi retiro total?', 'es necesario tener algun documento que justique el retiro total?', 'si no tengo algun documento para justificar mi retiro total puedo solicitarlo?']
ms: 1
idx_tok_wnm: [1, 5]
idx_elem: 0
ms: 2
idx_tok_wnm: [5]
idx_elem: 2
ms: 1
idx_tok_wnm: [1]
idx_elem: 0
ms: 1
idx_tok_wnm: []
idx_elem: 4
ms: 1
idx_tok_wnm: []
idx_elem: 2
ms: 1
idx_tok_wnm: []
idx_el

{'intent_name': ['retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'procedimiento_tramites_fechas',
  'procedimient

In [100]:
dict_data

{'intent_name': ['retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'retiro_total__requisitos_documentos_justificacion',
  'procedimiento_tramites_fechas',
  'procedimient

In [101]:
df_data =  pd.DataFrame(dict_data)
df_data.to_csv('test_aug_misspellings.csv', index = False)
df_data

,intent_name,examples,label
0,retiro_total__requisitos_documentos_justificacion,debo justificar la razon del retiro toal?,25
1,retiro_total__requisitos_documentos_justificacion,debo presentar documentos para justificar los ...,25
2,retiro_total__requisitos_documentos_justificacion,¿es necesario justificar con documentos el mot...,25
3,retiro_total__requisitos_documentos_justificacion,es necesario presentar documentos para justifi...,25
4,retiro_total__requisitos_documentos_justificacion,de que manera se tiene que justificar el retir...,25
...,...,...,...
836,constancia_matricula__pago,quiero solicitar una constancia de matricula c...,37
837,constancia_matricula__pago,la constansia de matricula tiene algun costo ?,37
838,constancia_matricula__pago,el tramite de la constansia de matricula seria...,37
839,constancia_matricula__pago,el trámite de la cconstancia de matricula es g...,37


In [102]:
df_data.tail(50)

,intent_name,examples,label
791,reserva_matricula__fechas,todavia puedo enviar mi reserva de matricula?,30
792,reserva_matricula__fechas,hasta que dias podria enviar la solicitud de r...,30
793,reserva_matricula__fechas,¿hasta que dias puedo enviar la solicitud de r...,30
794,reserva_matricula__fechas,si donde puedo encontrar las fechas para hacer...,30
795,reserva_matricula__fechas,hasta cuando puedo tramitar la reserva de matr...,30
796,reserva_matricula__fechas,¿ hasta q dias puedo enviar la solisitud de re...,30
797,reserva_matricula__fechas,si donde puedo encontrar las fechas para acer ...,30
798,reserva_matricula__fechas,todavia puedo enviar mi resreva de matricula ?,30
799,reserva_matricula__formato_solicitud,puedes enviarme el modelo para la solicitud de...,31
800,reserva_matricula__formato_solicitud,me un puedes un modelo para la solicitud de re...,31
